In [7]:
! pip uninstall sentencepiece -y
! pip install sentencepiece

Found existing installation: sentencepiece 0.1.97
Uninstalling sentencepiece-0.1.97:
  Successfully uninstalled sentencepiece-0.1.97
  Using cached sentencepiece-0.1.97-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


In [9]:
# Conda 환경이여서 conda install
!conda install -c powerai sentencepiece -y
# python 환경이면 pip install
!pip install sentencepiece -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda/envs/sts

  added / updated specs:
    - sentencepiece


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |              mkl           2 KB
    abseil-cpp-20211102.0      |       hd4dd3e8_0        1020 KB
    absl-py-1.3.0              |   py37h06a4308_0         166 KB
    aiohttp-3.8.1              |   py37h7f8727e_1         514 KB
    aiosignal-1.2.0            |     pyhd3eb1b0_0          12 KB
    astunparse-1.6.3           |             py_0          17 KB
    async-timeout-4.0.2        |   py37h06a4308_0          11 KB
    asynctest-0.13.0           |             py_0          26 KB
    attrs-21.4.

In [1]:
from tqdm.auto import tqdm
from transformers import pipeline
import pytorch_lightning as pl
import pandas as pd
import os
os.chdir("/opt/ml")

model_to_en = pipeline('translation',model='Helsinki-NLP/opus-mt-ko-en')
model_to_kr = pipeline('translation',model='Helsinki-NLP/opus-mt-tc-big-en-ko')
df = pd.read_csv('./data/train.csv')
temp = df.iloc[1,[2,3]]
temp_en = model_to_en(list(temp))
asd = [list(x.values())[0] for x in temp_en]
temp_kr = model_to_kr(asd[1])

/opt/conda/envs/sts/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/sts/lib/python3.7/site-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
temp_kr

[{'translation_text': '위스키, 피, US 9001 호텔 에서 적절한 well.'}]

In [8]:
temp = df.iloc[1,[2,3]]
temp_en = model_to_en(list(temp))
asd = [list(x.values())[0] for x in temp_en]
temp_kr = model_to_kr(asd[1])

In [29]:
temp_en[0][0]['translation_text']

"It's got thrills, it's got twists, it's not like any other Korean movie crap."

In [48]:
def fun(text):
    return text[0]['translation_text']

In [36]:
temp_en1 = df[:10].sentence_1.apply(model_to_en)
temp_en1 = temp_en1.apply(fun)

In [37]:
temp_en2 = df[:10].sentence_2.apply(model_to_en)
temp_en2 = temp_en2.apply(fun)

In [40]:
temp_en = pd.concat([temp_en1, temp_en2],axis=1)

In [47]:
temp_en

,sentence_1,sentence_2
0,"It's got thrills, it's got twists, it's not li...","There's a twist, there's love, there's fun."
1,I don't have access;,"Oh, Access, I don't have the authority."
2,Please change the housing conditions.,Please change the housing standards.
3,It was nice to meet you for the first time aft...,It was really nice to see you in real life.
4,You're so proud of me!,I'll see you in real life.
5,"Oh, my God.","Oh, my God, it's Caesars. It's a ghost fan."
6,I don't know if I'm gonna get cancer.,"Even with cancer, the sky is black."
7,I've never seen such cute rats before.,I've never seen such a boring horror movie...
8,It's the most urgent problem!,The most urgent thing is the care of the nursery!
9,Please take care of the change in the creamhouse.,Let us take care of the creamhouse.


In [45]:
def re_trans_to_ko(s):
  translator = Translator()
  return (translator.translate(s, dest="ko").text)

In [46]:
temp_en.sentence_1.apply(re_trans_to_ko)

0    스릴이 있고, 비틀림이 있고, 다른 한국 영화 쓰레기와는 다릅니다.
1                            나는 접근 할 수 없다.
2                           주택 조건을 변경하십시오.
3                   사망 후 처음으로 만나서 반가 웠습니다.
4                         당신은 너무 자랑 스럽습니다!
5                                     맙소사.
6                          내가 암에 걸릴지 모르겠다.
7                 나는 전에 그런 귀여운 쥐를 본 적이 없다.
8                            가장 시급한 문제입니다!
9                      크림 하우스의 변화를 처리하십시오.
Name: sentence_1, dtype: object

In [ ]:
def dic_to_txt(text):
    return text[0]['translation_text']

In [49]:
df.sentence_1 = df.sentence_1.apply(model_to_en).apply(dic_to_txt)
df.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,"It's got thrills, it's got twists, it's not li...","반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,I don't have access;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,Please change the housing conditions.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,It was nice to meet you for the first time aft...,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,You're so proud of me!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0


In [53]:
df.iloc[:20, [2,3]]

,sentence_1,sentence_2
0,"It's got thrills, it's got twists, it's not li...","반전도 있고,사랑도 있고재미도있네요."
1,I don't have access;,"오, 액세스 권한이 없다고 합니다."
2,Please change the housing conditions.,주택청약 무주택기준 변경해주세요.
3,It was nice to meet you for the first time aft...,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.
4,You're so proud of me!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!
5,"Oh, my God.",오 마이 갓 지저스 스크론 이스트 팬
6,I don't know if I'm gonna get cancer.,암만 찍어도 하늘은 까맣다.. ㅠㅠ
7,I've never seen such cute rats before.,이렇게 지겨운 공포영화는 처음..
8,It's the most urgent problem!,가장 시급한 것이 신생아실 관리입니다!!!
9,Please take care of the change in the creamhouse.,크림하우스 환불조치할 수 있도록해주세여


In [56]:
from googletrans import Translator

In [62]:
def re_trans_to_ko(s):
  translator = Translator()
  translator.raise_Exception = True
  return (translator.translate(s, dest="ko").text)

In [58]:
df.sentence_1 = df.sentence_1.apply(re_trans_to_ko)

AttributeError: 'Translator' object has no attribute 'raise_Exception'

In [64]:
df[:10].sentence_1.apply(re_trans_to_ko)

Exception: Unexpected status code "429" from ('translate.google.com',)

In [ ]:
df.sentence_2 = df.sentence_2.apply(model_to_en).apply(dic_to_txt).apply(re_trans_to_ko)
df.head()

In [ ]:
origin_df = pd.read_csv('./data/train.csv')

In [ ]:
df_trans_aug = pd.concat([origin_df, df])
print(df_trans_aug.shape) # 18648
print(df_trans_aug.head())
df_trans_aug.tail()

In [ ]:
df_trans_aug.to_csv('df_trans_aug.csv',header=False, index=False)

In [65]:
df.sentence_2 = df.sentence_2.apply(model_to_en).apply(dic_to_txt)

In [ ]:
df.to_csv('df_trans_en.csv',header=False, index=False)